In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append(r'C:/Users/vshnu/OneDrive/Desktop/Vishnuperson/RE/aspire_')
from connector import data_fetch_v2

In [ ]:
billing_data = f"""
select billing_cycle, user_id, payment_min
from billing_info """
flag,billing_data_df, error_msg = data_fetch_v2(billing_data, host="13.126.97.63")

In [ ]:
prod = f"""
select user_id, prod_group from sanction """
flag,prod_df, error_msg = data_fetch_v2(prod, host="13.126.97.63")

In [13]:
billing_data_df1 = billing_data_df.merge(prod_df, on = 'user_id', how='left')

In [14]:
billing_data_df1.head(4)

,billing_cycle,user_id,payment_min,prod_group
0,202104,41c99020-32a6-4911-afd3-313f08de9e03,1609.40,4M
1,202104,1f2b7fe3-77e6-43e3-96aa-7cf85f7f838e,9469.20,4M
2,202104,d0c030ac-8e0a-42d0-8d8e-9fd22f95b5ae,7544.40,4M
3,202104,fc471518-2940-41f2-9efb-470ed0d3a90c,7246.45,4M


Payments

In [ ]:
pay_now = f"""
select user_id, created_time, amount from pay_now_summary where status = 'success' """
flag,pay_now_df, error_msg = data_fetch_v2(pay_now, host="13.126.97.63")

In [ ]:
repay_details = f"""
select user_id, payment_time, amount from repayment_details where 
event in ('easebuzz_pay_back_link','nach_pull') """
flag,repay_details_df, error_msg = data_fetch_v2(repay_details, host="13.126.97.63")

In [ ]:
print(pay_now_df.head(5))
print(repay_details_df.head(5))

In [ ]:
pay_now_df['payment_cycle'] = pay_now_df['created_time'].dt.strftime('%Y%m')
repay_details_df['payment_cycle'] = repay_details_df['payment_time'].dt.strftime('%Y%m')

In [27]:
pay_now_df = pay_now_df.drop(columns = ['created_time'])
repay_details_df = repay_details_df.drop(columns = ['payment_time'])

In [30]:
payments_df = pd.concat([pay_now_df,repay_details_df])

In [32]:
payments_df.head(5)

,user_id,amount,payment_cycle
0,bb7dfe91-6fe0-4159-bd5c-d83b3544b388,150.0,202101
1,41c99020-32a6-4911-afd3-313f08de9e03,100.0,202101
2,41c99020-32a6-4911-afd3-313f08de9e03,96.0,202101
3,bb7dfe91-6fe0-4159-bd5c-d83b3544b388,100.0,202102
4,a54f4f44-9860-44df-94cb-91ac7798f1ed,149.0,202102


In [37]:
payments_df_ = payments_df.groupby(['user_id', 'payment_cycle'])['amount'].sum().reset_index()


In [38]:
type(payments_df_)

pandas.core.frame.DataFrame

In [39]:
bounce = billing_data_df1.merge(payments_df_, left_on = ('user_id','billing_cycle'), right_on = ('user_id','payment_cycle'), how = 'left' )

In [44]:
bounce.head(5)

,billing_cycle,user_id,payment_min,prod_group,payment_cycle,amount,fl_bounce
0,202104,41c99020-32a6-4911-afd3-313f08de9e03,1609.40,4M,202104,1762.15,0
1,202104,1f2b7fe3-77e6-43e3-96aa-7cf85f7f838e,9469.20,4M,NaN,NaN,1
2,202104,d0c030ac-8e0a-42d0-8d8e-9fd22f95b5ae,7544.40,4M,NaN,NaN,1
3,202104,fc471518-2940-41f2-9efb-470ed0d3a90c,7246.45,4M,NaN,NaN,1
4,202104,bb7dfe91-6fe0-4159-bd5c-d83b3544b388,4879.00,4M,202104,6151.25,0


In [43]:
bounce["fl_bounce"] = np.where((bounce["payment_min"] == 0) | (bounce["payment_min"] < bounce["amount"] + 10),0,1)

In [ ]:
pivot = bounce.groupby(['billing_cycle','fl_bounce'])['user_id'].count().unstack(fill_value=0)


In [ ]:
pivot.head(2)

In [58]:
pivot["total"] = pivot[0]+pivot[1]

In [61]:
pivot["bounce_percent"] = round((pivot[0]/pivot["total"])*100,2)

In [63]:
pivot["bounce_percent"] = pivot["bounce_percent"].apply(lambda x: f"{x}%")

In [ ]:
pivot['bounce_percent'] = round((pivot['fl_bounce'].eq(1).sum() / pivot['fl_bounce'].count()) * 100, 2)


In [53]:
# Calculate bounce percentage within each billing_cycle
pivot['bounce_percent'] = pivot.groupby('billing_cycle')['fl_bounce'].transform(lambda x: round((x.eq(1).sum() / len(x)) * 100, 2))
